In [2]:
import pandas as pd
import numpy as np
import joblib as jb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing  import LabelEncoder

In [3]:
data=pd.read_csv("rawdata.csv")
data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0


In [4]:
data['TX_DATETIME']=pd.to_datetime(data['TX_DATETIME'])
data['T_YEAR']=data['TX_DATETIME'].dt.year
data['T_MONTH']=data['TX_DATETIME'].dt.month
data['T_DAY']=data['TX_DATETIME'].dt.day



In [5]:
data.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,T_YEAR,T_MONTH,T_DAY
0,0,2018-04-01 00:00:31,596,3156,57.16,31,0,0,0,2018,4,1
1,1,2018-04-01 00:02:10,4961,3412,81.51,130,0,0,0,2018,4,1
2,2,2018-04-01 00:07:56,2,1365,146.00,476,0,0,0,2018,4,1
3,3,2018-04-01 00:09:29,4128,8737,64.49,569,0,0,0,2018,4,1
4,4,2018-04-01 00:10:34,927,9906,50.99,634,0,0,0,2018,4,1


In [6]:
data=data.drop(columns=['TX_DATETIME','TX_FRAUD_SCENARIO','TX_TIME_SECONDS','TX_TIME_DAYS'])

In [7]:
data.insert(2,'TX_YEAR',data['T_YEAR'])
data.insert(3,'TX_MONTH',data['T_MONTH'])
data.insert(4,'TX_DAY',data['T_DAY'])



In [8]:
data=data.drop(columns=['T_YEAR','T_MONTH','T_DAY'])

In [11]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   TRANSACTION_ID  int64  
 1   CUSTOMER_ID     int64  
 2   TX_YEAR         int32  
 3   TX_MONTH        int32  
 4   TX_DAY          int32  
 5   TERMINAL_ID     int64  
 6   TX_AMOUNT       float64
 7   TX_FRAUD        int64  
dtypes: float64(1), int32(3), int64(4)
memory usage: 87.0 MB


In [12]:
data['TX_YEAR']=data['TX_YEAR'].astype('int64')
data['TX_MONTH']=data['TX_MONTH'].astype('int64')
data['TX_DAY']=data['TX_DAY'].astype('int64')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754155 entries, 0 to 1754154
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   TRANSACTION_ID  int64  
 1   CUSTOMER_ID     int64  
 2   TX_YEAR         int64  
 3   TX_MONTH        int64  
 4   TX_DAY          int64  
 5   TERMINAL_ID     int64  
 6   TX_AMOUNT       float64
 7   TX_FRAUD        int64  
dtypes: float64(1), int64(7)
memory usage: 107.1 MB


In [ ]:
data['AMOUNT_FRAUD'] = data['TX_AMOUNT'] > 220
fraud_terminal=data[data['TX_FRAUD']==1]['TERMINAL_ID'].value_counts()
data["FRAUD_TERMINAL_SCORE"]=data['TERMINAL_ID'].map(fraud_terminal)
data['FRAUD_TERMINAL_SCORE']=data['FRAUD_TERMINAL_SCORE'].fillna(value=0).astype('int64')

In [16]:

dict=fraud_terminal.to_dict()
jb.dump(dict,r"fraudterminaldict.pkl")


['fraudterminaldict.pkl']

In [17]:
data['AMOUNT_FRAUD']=LabelEncoder().fit_transform(data['AMOUNT_FRAUD'])


In [18]:
data

,TRANSACTION_ID,CUSTOMER_ID,TX_YEAR,TX_MONTH,TX_DAY,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,AMOUNT_FRAUD,FRAUD_TERMINAL_SCORE
0,0,596,2018,4,1,3156,57.16,0,0,17
1,1,4961,2018,4,1,3412,81.51,0,0,1
2,2,2,2018,4,1,1365,146.00,0,0,1
3,3,4128,2018,4,1,8737,64.49,0,0,0
4,4,927,2018,4,1,9906,50.99,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,161,2018,9,30,655,54.24,0,0,0
1754151,1754151,4342,2018,9,30,6181,1.23,0,0,2
1754152,1754152,618,2018,9,30,1502,6.62,0,0,0
1754153,1754153,4056,2018,9,30,3067,55.40,0,0,0


In [ ]:
data.insert(8,"ABOVE220",data['AMOUNT_FRAUD'])
data.insert(9,'FRAUD_SCORE',data['FRAUD_TERMINAL_SCORE'])
data=data.drop(columns=['AMOUNT_FRAUD',"FRAUD_TERMINAL_SCORE"])
data.insert(10,'FRAUD',data['TX_FRAUD'])
del data['TX_FRAUD']

In [23]:
data

,TRANSACTION_ID,CUSTOMER_ID,TX_YEAR,TX_MONTH,TX_DAY,TERMINAL_ID,TX_AMOUNT,ABOVE220,FRAUD_SCORE,FRAUD
0,0,596,2018,4,1,3156,57.16,0,17,0
1,1,4961,2018,4,1,3412,81.51,0,1,0
2,2,2,2018,4,1,1365,146.00,0,1,0
3,3,4128,2018,4,1,8737,64.49,0,0,0
4,4,927,2018,4,1,9906,50.99,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,161,2018,9,30,655,54.24,0,0,0
1754151,1754151,4342,2018,9,30,6181,1.23,0,2,0
1754152,1754152,618,2018,9,30,1502,6.62,0,0,0
1754153,1754153,4056,2018,9,30,3067,55.40,0,0,0


In [24]:
data.to_csv(r"C:\Users\Avijit\Desktop\fraud_detection\src\model\processdata.csv",index=False)